## 2. PyLipID Analysis

Perform the PyLipID analysis of the simulations, extracting:
- Occupancy
- Lipid count
- koff / Residence Time
- Bound poses

In [1]:
#Load required libraries
import MDAnalysis as mda
import pylipid
from pylipid.api import LipidInteraction
from pylipid.util import check_dir
import os

In [2]:
#Assign variables
trpv = 1
resi_offset = 115 #Residue offset between the original .pdb and the .gro file
stride = 10

folder = f"../Simulations/TRPV{trpv}"

trajfile_list = [f"{folder}/replica_{r}/Analysis/analysis_centered_step7_production.xtc" for r in [1,2,3]]
topfile_list = [f"{folder}/replica_{r}/Analysis/analysis_step7_coord.gro" for r in [1,2,3]]
                                                                                        

cutoffs = [0.475, 0.7]
lipid = "PAP6"
lipid_atoms = {                
                "POPA":["PO4"],
                "PAP1":["PO4", "C1", "C2", "C3", "P1"],
                "PAP6":["PO4", "C1", "C2", "C3", "P4", "P5"],
                "CHOL":None,
                "POPC":["PO4", "NC3"],
                "PAPC":["PO4", "NC3"],
                "POPE":["PO4", "NH3"],
                "DIPE":["PO4", "NH3"],
                "DPSM":["PO4", "NC3"],
                "PAPS":["PO4", "CNO"]
                }

nprot = 1
timeunit = "us"
fig_format = "svg"

save_dir = f"{folder}/PyLipid"

num_cpus = 32

binding_site_size = 4
save_pose_format = "gro"
n_top_poses = 3
n_clusters = "auto"

save_pose_traj = True
save_pose_traj_format = "xtc"


pdb_file_to_map = None 


In [3]:
#Create PyLipid directory
try:
    os.mkdir(f"{folder}/PyLipid")
    print(f"Folder 'PyLipid' created in {folder}")
except FileExistsError:
    print(f"Could not create folder. Folder 'PyLipid' already exists in {folder}")

Could not create folder. Folder 'PyLipid' already exists in ../Simulations/TRPV1


In [4]:
for lipid, atoms in lipid_atoms.items():
    print(f"Analyzing trajectories for {lipid}")
    # initialize
    li = LipidInteraction(trajfile_list, topfile_list=topfile_list, cutoffs=cutoffs, 
                        lipid=lipid, 
                        lipid_atoms=atoms,
                        resi_offset=resi_offset,
                        stride=stride,
                        nprot=nprot, 
                        save_dir=save_dir)

    #Compute contacts
    li.collect_residue_contacts()

    #Compute residue data (Duration, Occupancy, LipidCount, koff)
    li.compute_residue_duration(residue_id=None)
    li.compute_residue_occupancy(residue_id=None)
    li.compute_residue_lipidcount(residue_id=None)
    li.show_stats_per_traj(write_log=True, print_log=True)
    li.compute_residue_koff(residue_id=None, plot_data=True, fig_close=True,
                            fig_format=fig_format, num_cpus=num_cpus)

    #Calculate binding sites data
    li.compute_binding_nodes(threshold=binding_site_size, print_data=False)
    if len(li.node_list) == 0:
        print("*"*50)
        print("No binding site detected! Skip analysis for binding sites.")
        print("*"*50)
    else:
        li.compute_site_duration(binding_site_id=None)
        li.compute_site_occupancy(binding_site_id=None)
        li.compute_site_lipidcount(binding_site_id=None)
        li.compute_site_koff(binding_site_id=None, plot_data=True, fig_close=True,
                            fig_format=fig_format, num_cpus=num_cpus)
        pose_traj, pose_rmsd_data = li.analyze_bound_poses(binding_site_id=None, pose_format=save_pose_format,
                                                        n_top_poses=n_top_poses, n_clusters=n_clusters,
                                                        fig_format=fig_format, num_cpus=num_cpus)
        # save pose trajectories
        if save_pose_traj:
            for bs_id in pose_traj.keys():
                pose_traj[bs_id].save("{}/Bound_Poses_{}/Pose_traj_BSid{}.{}".format(li.save_dir, li.lipid, bs_id,
                                                                            save_pose_traj_format))
        del pose_traj  # save memory space
        surface_area_data = li.compute_surface_area(binding_site_id=None, radii=None, fig_format=fig_format)
        data_dir = check_dir(li.save_dir, "Dataset_{}".format(li.lipid))
        pose_rmsd_data.to_csv("{}/Pose_RMSD_data.csv".format(data_dir), index=False, header=True)
        surface_area_data.to_csv("{}/Surface_Area_data.csv".format(data_dir), index=True, header=True)
        li.write_site_info(sort_residue="Residence Time")

    #### write and save data
    for item in ["Dataset", "Duration", "Occupancy", "Lipid Count", "CorrCoef"]:
        li.save_data(item=item)
    for item in ["Residence Time", "Duration", "Occupancy", "Lipid Count"]:
        li.save_coordinate(item=item)
    for item in ["Residence Time", "Duration", "Occupancy", "Lipid Count"]:
        li.plot(item=item, fig_close=True, fig_format=fig_format)
        li.plot_logo(item=item, fig_close=True, fig_format=fig_format)

    li.dataset.to_csv(f"{save_dir}/TRPV{trpv}_full_dataset.csv")


Analyzing trajectories for CHOL


COLLECT INTERACTIONS FROM TRAJECTORIES:   0%|          | 0/1 [01:11<?, ?it/s]


KeyboardInterrupt: 